In [1]:
import requests
import json
import datetime
from datetime import timedelta
import pytz
from pytz import timezone
import pandas as pd
from bisect import bisect_left

# Decoding된 일반인증키 사용
key = 'YOUR_KEY'

In [2]:
now = datetime.datetime.now(timezone('Asia/Seoul'))

vilage_time = [datetime.datetime(now.year, now.month, now.day, 11, 10, tzinfo=pytz.timezone('Asia/Seoul')) + timedelta(hours=3) * i for i in range(-4, 5)]
vilage_time = vilage_time[bisect_left(vilage_time, now) - 1]

srt_time = now

base_date_vilage = f'{vilage_time.year}{vilage_time.month:02d}{vilage_time.day:02d}'
base_time_vilage = f'{vilage_time.hour:02d}00'

if now.minute <= 45:
    srt_time = now - timedelta(minutes=45)

base_date_srt = f'{srt_time.year}{srt_time.month:02d}{srt_time.day:02d}'
base_time_srt = f'{srt_time.hour:02d}00'

In [3]:
# 단기예보
VilageFcst_url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst'
params ={'serviceKey' : key, 'pageNo' : '1', 'numOfRows' : '1000', 
         'dataType' : 'JSON', 'base_date' : base_date_vilage, 'base_time' : base_time_vilage, 
         'nx' : '59', 'ny' : '122' }

response = requests.get(VilageFcst_url, params=params)
jsn = json.loads(response.content)


In [4]:
item = jsn['response']['body']['items']['item']
mdf = pd.DataFrame(item)
mdf.head()

,baseDate,baseTime,category,fcstDate,fcstTime,fcstValue,nx,ny
0,20230425,1100,TMP,20230425,1200,12,59,122
1,20230425,1100,UUU,20230425,1200,-0.1,59,122
2,20230425,1100,VVV,20230425,1200,1.9,59,122
3,20230425,1100,VEC,20230425,1200,174,59,122
4,20230425,1100,WSD,20230425,1200,1.9,59,122


In [5]:
mdf['baseTime'] = mdf['baseDate'] +  mdf['baseTime']
mdf['fcstTime'] = mdf['fcstDate'] +  mdf['fcstTime']
mdf['baseTime'] = pd.to_datetime(mdf['baseTime'])
mdf['fcstTime'] = pd.to_datetime(mdf['fcstTime'])

mdf = mdf.drop(columns=['baseDate', 'fcstDate'])
mdf = mdf.rename(columns={'baseTime': '발표시각', 'fcstTime': '예보시각', 
                          'nx': '예보지점X', 'ny': '예보지점Y', 
                          'category': '예보구분'})
mdf = mdf.pivot_table(values='fcstValue', 
                      index=['발표시각', '예보지점X', '예보지점Y', '예보시각'], 
                      columns='예보구분', aggfunc='first')
mdf = mdf.rename(columns={'PCP': '1시간 강수량(1mm)', 'POP': '강수확률(%)', 
                          'PTY': '강수형태', 'REH': '습도(%)', 'SKY': '하늘상태', 
                          'SNO': '1시간 신적설(1cm)', 'TMN': '일 최저기온(℃)', 
                          'TMX': '일 최고기온(℃)', 'TMP': '1시간 기온(℃)', 
                          'UUU': '풍속(동서성분)(m/s)', 'VVV': '풍속(남북성분)(m/s)', 
                          'WAV': '파고(M)', 'VEC': '풍향(deg)', 'WSD': '풍속(m/s)'})
mdf['강수형태'].replace({'0': '없음', '1': '비', '2': '비/눈', 
                     '3': '눈', '4': '소나기'}, inplace=True)
mdf['하늘상태'].replace({'1': '맑음', '3': '구름많음', '4': '흐림'}, inplace=True)
mdf

예보구분                                                1시간 강수량(1mm) 강수확률(%) 강수형태  \
발표시각                예보지점X 예보지점Y 예보시각                                            
2023-04-25 11:00:00 59    122   2023-04-25 12:00:00         강수없음      60    비   
                                2023-04-25 13:00:00         강수없음      60    비   
                                2023-04-25 14:00:00         강수없음      60    비   
                                2023-04-25 15:00:00         강수없음      30   없음   
                                2023-04-25 16:00:00         강수없음      30   없음   
...                                                          ...     ...  ...   
                                2023-04-27 20:00:00         강수없음       0   없음   
                                2023-04-27 21:00:00         강수없음       0   없음   
                                2023-04-27 22:00:00         강수없음       0   없음   
                                2023-04-27 23:00:00         강수없음       0   없음   
                                2023-04-28 00:00:00         강수없음       0   없음   

예보구분                                                습도(%) 하늘상태 1시간 신적설(1cm)  \
발표시각                예보지점X 예보지점Y 예보시각                                          
2023-04-25 11:00:00 59    122   2023-04-25 12:00:00    60   흐림         적설없음   
                                2023-04-25 13:00:00    60   흐림         적설없음   
                                2023-04-25 14:00:00    60   흐림         적설없음   
                                2023-04-25 15:00:00    55   흐림         적설없음   
                                2023-04-25 16:00:00    55   흐림         적설없음   
...                                                   ...  ...          ...   
                                2023-04-27 20:00:00    65   맑음         적설없음   
                                2023-04-27 21:00:00    70   맑음         적설없음   
                                2023-04-27 22:00:00    70   맑음         적설없음   
                                2023-04-27 23:00:00    70   맑음         적설없음   
                                2023-04-28 00:00:00    70   맑음         적설없음   

예보구분                                                일 최저기온(℃) 1시간 기온(℃)  \
발표시각                예보지점X 예보지점Y 예보시각                                      
2023-04-25 11:00:00 59    122   2023-04-25 12:00:00       NaN        12   
                                2023-04-25 13:00:00       NaN        13   
                                2023-04-25 14:00:00       NaN        13   
                                2023-04-25 15:00:00       NaN        13   
                                2023-04-25 16:00:00       NaN        13   
...                                                       ...       ...   
                                2023-04-27 20:00:00       NaN        14   
                                2023-04-27 21:00:00       NaN        14   
                                2023-04-27 22:00:00       NaN        13   
                                2023-04-27 23:00:00       NaN        12   
                                2023-04-28 00:00:00       NaN        11   

예보구분                                                일 최고기온(℃) 풍속(동서성분)(m/s)  \
발표시각                예보지점X 예보지점Y 예보시각                                          
2023-04-25 11:00:00 59    122   2023-04-25 12:00:00       NaN          -0.1   
                                2023-04-25 13:00:00       NaN          -0.2   
                                2023-04-25 14:00:00       NaN           0.1   
                                2023-04-25 15:00:00      14.0           0.4   
                                2023-04-25 16:00:00       NaN           1.2   
...                                                       ...           ...   
                                2023-04-27 20:00:00       NaN           1.1   
                                2023-04-27 21:00:00       NaN           1.1   
                                2023-04-27 22:00:00       NaN           0.5   
                                2023-04-27 23:00:00       NaN           0

In [6]:
# 초단기예보
UltraSrtFcst_url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtFcst'
params ={'serviceKey' : key, 'pageNo' : '1', 'numOfRows' : '1000', 
         'dataType' : 'JSON', 'base_date' : base_date_srt, 'base_time' : base_time_srt, 
         'nx' : '59', 'ny' : '122' }

response = requests.get(UltraSrtFcst_url, params=params)
jsn = json.loads(response.content)


In [7]:
item = jsn['response']['body']['items']['item']
mdf = pd.DataFrame(item)
mdf.head()

,baseDate,baseTime,category,fcstDate,fcstTime,fcstValue,nx,ny
0,20230425,1230,LGT,20230425,1300,0,59,122
1,20230425,1230,LGT,20230425,1400,0,59,122
2,20230425,1230,LGT,20230425,1500,0,59,122
3,20230425,1230,LGT,20230425,1600,0,59,122
4,20230425,1230,LGT,20230425,1700,0,59,122


In [8]:
mdf['baseTime'] = mdf['baseDate'] +  mdf['baseTime']
mdf['fcstTime'] = mdf['fcstDate'] +  mdf['fcstTime']
mdf['baseTime'] = pd.to_datetime(mdf['baseTime'])
mdf['fcstTime'] = pd.to_datetime(mdf['fcstTime'])

mdf = mdf.drop(columns=['baseDate', 'fcstDate'])
mdf = mdf.rename(columns={'baseTime': '발표시각', 'fcstTime': '예보시각', 
                          'nx': '예보지점X', 'ny': '예보지점Y', 
                          'category': '예보구분'})
mdf = mdf.pivot_table(values='fcstValue', 
                      index=['발표시각', '예보지점X', '예보지점Y', '예보시각'], 
                      columns='예보구분', aggfunc='first')

mdf = mdf.rename(columns={'T1H': '기온(℃)', 'RN1': '1시간 강수량(1mm)', 
                          'PTY': '강수형태', 'REH': '습도(%)', 'SKY': '하늘상태', 
                          'LGT': '낙뢰(kA)', 'UUU': '풍속(동서성분)(m/s)', 
                          'VVV': '풍속(남북성분)(m/s)', 
                          'VEC': '풍향(deg)', 'WSD': '풍속(m/s)'})
mdf['강수형태'].replace({'0': '없음', '1': '비', '2': '비/눈', '3': '눈', 
                     '5': '빗방울', '6': '빗방울눈날림', '7': '눈날림'}, inplace=True)
mdf['하늘상태'].replace({'1': '맑음', '3': '구름많음', '4': '흐림'}, inplace=True)
mdf

예보구분                                                낙뢰(kA) 강수형태 습도(%)  \
발표시각                예보지점X 예보지점Y 예보시각                                    
2023-04-25 12:30:00 59    122   2023-04-25 13:00:00      0    비    60   
                                2023-04-25 14:00:00      0    비    60   
                                2023-04-25 15:00:00      0    비    55   
                                2023-04-25 16:00:00      0    비    55   
                                2023-04-25 17:00:00      0    비    60   
                                2023-04-25 18:00:00      0    비    60   

예보구분                                                1시간 강수량(1mm) 하늘상태 기온(℃)  \
발표시각                예보지점X 예보지점Y 예보시각                                          
2023-04-25 12:30:00 59    122   2023-04-25 13:00:00        1.0mm   흐림    12   
                                2023-04-25 14:00:00        1.0mm   흐림    12   
                                2023-04-25 15:00:00         강수없음   흐림    12   
                                2023-04-25 16:00:00         강수없음   흐림    13   
                                2023-04-25 17:00:00        1.0mm   흐림    12   
                                2023-04-25 18:00:00        1.0mm   흐림    11   

예보구분                                                풍속(동서성분)(m/s) 풍향(deg)  \
발표시각                예보지점X 예보지점Y 예보시각                                        
2023-04-25 12:30:00 59    122   2023-04-25 13:00:00          -1.6     172   
                                2023-04-25 14:00:00          -1.1     195   
                                2023-04-25 15:00:00          -0.4     229   
                                2023-04-25 16:00:00           0.8     251   
                                2023-04-25 17:00:00           1.9     270   
                                2023-04-25 18:00:00           2.3     282   

예보구분                                                풍속(남북성분)(m/s) 풍속(m/s)  
발표시각                예보지점X 예보지점Y 예보시각                                       
2023-04-25 12:30:00 59    122   2023-04-25 13:00:00           0.7       1  
                                2023-04-25 14:00:00           0.4       1  
                                2023-04-25 15:00:00           0.1       1  
                                2023-04-25 16:00:00           0.3       1  
                                2023-04-25 17:00:00             0       2  
                                2023-04-25 18:00:00          -0.4       2